In [1]:
import os
import shutil

# = = = = = = = = = = = = = = =

if os.path.exists('./file/5.collate'):
    shutil.rmtree('./file/5.collate')

os.makedirs('./file/5.collate')

# 1. Json

In [2]:
import pandas as pd

from tqdm import tqdm
import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.info'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/4.info/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    
    # = = = = = = = = = = = = = = =

    list_column, list_column_price, list_column_note, list_column_note_2, list_column_specification = input_.columns.tolist(), ['Price'], ['Note 1'], ['Note 2'], []

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
        df_temp, dict_price, dict_note, dict_note_2, dict_dict = pd.DataFrame([input_.iloc[i]]).reset_index(drop=True), json.loads(input_.loc[i, 'Json_Price']), json.loads(input_.loc[i, 'Json_Note 1']), json.loads(input_.loc[i, 'Json_Note 2']), json.loads(input_.loc[i, 'Json_Specification'])

        df_temp.loc[0, 'Price'] = dict_price['0']

        dict_price = dict_price.get('1', {})
        for key, value in dict_price.items():
            key = f'Price-{int(key) + 2}'

            if key not in list_column_price:
                list_column_price.append(key)
                
            df_temp.loc[0, key] = value
        
        # = = = = = = = = = = = = = = =

        if dict_note:
            for key, value in dict_note.items():
                if key == '0':
                    df_temp.loc[0, 'Note 1'] = dict_note['0']
                else:
                    key = f'Note 1-{int(key) + 1}'
    
                    if key not in list_column_note:
                        list_column_note.append(key)
                        
                    df_temp.loc[0, key] = value
        else:
            df_temp.loc[0, 'Note 1'] = ''

        # = = = = = = = = = = = = = = =

        if dict_note_2:
            for key, value in dict_note_2.items():
                if key == '0':
                    df_temp.loc[0, 'Note 2'] = dict_note_2['0']
                else:
                    key = f'Note 2-{int(key) + 1}'
    
                    if key not in list_column_note_2:
                        list_column_note_2.append(key)
                        
                    df_temp.loc[0, key] = value
        else:
            df_temp.loc[0, 'Note 2'] = ''

        # = = = = = = = = = = = = = = =
        
        for dict_ in dict_dict.values():
            if dict_['0'] in list_column:
                dict_['0'] = f'''{dict_['0']} (Specification)'''

            if dict_['0'] not in list_column_specification:
                list_column_specification.append(dict_['0'])
                
            df_temp.loc[0, dict_['0']] = dict_.get('1', 'N/A')
            
    
        list_df.append(df_temp)
    
    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output[list_column[:8] + list_column_price + list_column_note + list_column_note_2 + sorted(list_column_specification) + list_column[8:]].to_excel(f'./file/5.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 271.30it/s]



crawler_2.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 657.51it/s]


Done ~


# 2. Compatible

In [3]:
import pandas as pd

from tqdm import tqdm

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
# = = = = = = = = = = = = = = =

for file in list_file:

    print(file)

    # = = = = = = = = = = = = = = =
    
    input_ = pd.read_excel(f'./file/3.vehicle/{file}',
                       header=0,
                       dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)
    input_['Vehicle No'] = input_['Vehicle No'].astype(int)
    input_['Year'] = input_['Year'].astype(int)

    # = = = = = = = = = = = = = = =

    input_2 = input_.drop_duplicates(['JOIN_MPNTCPC'],
                                     keep='first',
                                     ignore_index=True)
    input_2 = input_2.sort_values(by=['No'],
                                  ascending=[True],
                                  ignore_index=True)

    # = = = = = = = = = = = = = = =

    list_df = []
    for i in tqdm(range(len(input_2)), desc='Progress', ncols=77):
        df_temp = input_[input_['JOIN_MPNTCPC'] == input_2.loc[i, 'JOIN_MPNTCPC']].reset_index(drop=True)

        dict_vehicle = {}
        for j in range(len(df_temp)):
            make, model, year = df_temp.loc[j, 'Make'], df_temp.loc[j, 'Model'], df_temp.loc[j, 'Year']
    
            if make not in dict_vehicle:
                dict_vehicle[make] = {model: [year]}
            elif model not in dict_vehicle[make]:
                dict_vehicle[make][model] = [year]
            elif year not in dict_vehicle[make][model]:
                dict_vehicle[make][model].append(year)
    
        # = = = = = = = = = = = = = = =

        list_vehicle = []
        for make in dict_vehicle:
            for model in dict_vehicle[make]:
                year_begin = min(dict_vehicle[make][model])
                year_end = max(dict_vehicle[make][model])
                list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

        # = = = = = = = = = = = = = = =

        df_temp = pd.DataFrame([{'No': input_2.loc[i, 'No'],
                                 'Part Number': input_2.loc[i, 'Part Number'],
                                 'JOIN_MPNTCPC': input_2.loc[i, 'JOIN_MPNTCPC'],
                                 'Vehicle': '\n'.join(sorted(list_vehicle))}])
        list_df.append(df_temp)

    # = = = = = = = = = = = = = = =

    output = pd.concat(list_df, ignore_index=True).fillna('')

    output = output.sort_values(by=['No'],
                                ascending=[True],
                                ignore_index=True)

    output.to_excel(f'./file/5.collate/{file}', index=False)

    # = = = = = = = = = = = = = = =

    print()

print('Done ~')

crawler_1-vehicle.xlsx


Progress: 100%|█████████████████████████████| 10/10 [00:00<00:00, 742.38it/s]



crawler_2-vehicle.xlsx


Progress: 100%|███████████████████████████████| 9/9 [00:00<00:00, 929.18it/s]



Done ~
